In [5]:
import torch
from torch import nn 
random_input = torch.randn(size=(100,16))

In [6]:
params = dict(n_tree=5,
     tree_depth=3,
     n_in_feature=5,
     tree_feature_rate=0.1,
     n_class=2,
     jointly_training=True)

In [11]:
n_in_feature = random_input.shape[1]
depth = 3 
used_feature_rate = 0.4
n_class = 2 
n_used_feature = int(n_in_feature*used_feature_rate)